In [1]:
import psycopg2 as pg
import random

# 获取DB连接对象
4个用户 \
    pm:      create 用户 数据库 \
    t_coder: 老师端, select 教师表, insert\update 成绩表 \
    s_coder: 学生端, select 学生表 课程表 成绩表 \ 
    affair:  教务处, insert 教师表 学生表 课程表 \

In [2]:
def get_conn(user='pm'):
    conn = pg.connect(database="db_school", 
                  user=user, 
                  password="123456twG", 
                  host="127.0.0.1", port="7654")
    return conn

# 从CSV文件导入
Day1, affair用户, 将student.tsv, teacher.tsv, course.tsv导入到数据库中

In [4]:
def insert_tsv(conn, table_name, tsv_path, sep='\t'):
    print("Opened database successfully")
    cur = conn.cursor()

    copy_query = f"COPY public.{table_name} FROM STDIN WITH (FORMAT CSV, HEADER TRUE, DELIMITER E'\\t', ENCODING 'UTF-8')"
    with open(tsv_path, 'r', encoding='utf-8') as file:
        cur.copy_expert(copy_query, file)
        print(f"Copy {tsv_path} ok")
    # 提交事务
    conn.commit()
    # 关闭数据库连接
    cur.close()
    conn.close()

In [ ]:
conn = get_conn(user='affair')
insert_tsv(conn, "teacher", '../data/teacher.tsv')

In [6]:
conn2 = get_conn(user='affair')
conn3 = get_conn(user='affair')
insert_tsv(conn2, "student", '../data/student.tsv')
insert_tsv(conn3, "course", '../data/course.tsv')

Opened database successfully
Copy ../data/student.tsv ok
Opened database successfully
Copy ../data/course.tsv ok


# 向score添加数据
Day2, t_coder用户，向score表插入数据

In [11]:
def insert_score(conn):
    student_id = range(20230001, 20230041)
    course_id = range(10, 15)
    cur = conn.cursor()
    
    insert_value = []
    for s_id in student_id:
        for c_id in course_id:
            random_score = random.randint(75, 95)
            insert_value.append((s_id, c_id, random_score))

    insert_sql = "INSERT INTO public.score (student_id, course_id, score) VALUES (%s, %s, %s)"
    cur.executemany(insert_sql, insert_value)
    conn.commit()
    cur.close()
    conn.close()
    

    
# 修改成绩
def update_score(conn, repete_times=10):
    cur = conn.cursor()    
        
    update_value = []
    for i in range(repete_times):
        s_id = random.randint(20230001, 20230040)
        c_id = random.randint(10, 14)
        det_score = random.randint(-5, 0)
        update_value.append((det_score, s_id, c_id))
        
    update_sql = "UPDATE public.score SET score = score+(%s) WHERE student_id=(%s) AND course_id=(%s) "
    cur.executemany(update_sql, update_value)
    conn.commit()
    cur.close()
    conn.close()

In [10]:
conn = get_conn(user='t_coder')
insert_score(conn)

In [12]:
conn_update_score = get_conn(user='t_coder')
update_score(conn_update_score)

# 查询teacher,student,score
Day1-5: t_coder查询teacher表，s_coder查询student表、course表 \
Day2: teacher查询score, 学生查询score

In [5]:
# 对表{table_name}指定{query_id}查询{repete_times}次
# 查询student teacher course都是调用这个函数
def select(conn, table_name, repete_times, query_id, id_range):
    cur = conn.cursor()    
    query_value = []    
    for t in range(repete_times):
        query_value.append((random.randint(id_range[0], id_range[1]),))
    query_sql = f"SELECT * FROM public.{table_name} WHERE {query_id}=(%s)"
    
    cur.executemany(query_sql, query_value)
    conn.commit()
    cur.close()
    conn.close()
    

# 学生查询课表
def select_course(conn, repete_times):
    cur = conn.cursor()        
    query_sql = "SELECT course_id, cname, name, point FROM public.course c, public.teacher t where c.teacher_id = t.teacher_id"    
    
    for i in range(repete_times):
        cur.execute(query_sql)
        result = cur.fetchall()
    conn.commit()
    cur.close()
    conn.close()


# 学生查询成绩
def select_score(conn):
    cur = conn.cursor()        
    query_sql = """SELECT s.student_id, s."name", c.cname, s2.score 
                    FROM public.score s2, public.course c, public.student s 
                    WHERE s2.course_id = c.course_id AND s2.student_id = s.student_id AND s.student_id=(%s)"""    
    query_value = [(i,) for i in range(20230001, 20230041)]
    
    cur.executemany(query_sql, query_value)    
    conn.commit()
    cur.close()
    conn.close()    


# 老师查询指定课程的所有成绩
def select_score_teacher(conn):
    cur = conn.cursor()    
    query_value = [(i,) for i in range(10, 15)]
    query_sql = "SELECT * FROM public.score WHERE course_id=(%s) ORDER BY score desc;"
    
    cur.executemany(query_sql, query_value)
    conn.commit()
    cur.close()
    conn.close()
    

执行查询学生信息、老师信息、课程信息脚本

In [6]:
conn_s = get_conn(user='s_coder')
conn_s2 = get_conn(user='s_coder')
conn_t = get_conn(user='t_coder')


s_repete = random.randint(70,100)
s2_repete = s_repete + random.randint(5,10)
t_repete = random.randint(10,30)

# s_coder查询学生信息
select(conn_s, "student", s_repete, "student_id", (20230001, 20230040))
# s_coder查询课程信息
select_course(conn_s2, s2_repete)
# t_coder查询教师信息
select(conn_t, "teacher", t_repete, "teacher_id", (101, 105))

查询成绩

In [15]:
conn_score = get_conn(user='s_coder')
conn_score_t = get_conn(user='t_coder')

# s_coder查询成绩
select_score(conn_score)
# t_coder按照课程号查询成绩
select_score_teacher(conn_score_t)